In [1]:
!pip install -q \
    transformers==4.31 \
    peft==0.5.0 \
    accelerate \
    bitsandbytes \
    trl==0.4.7 \
    datasets \
    evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 4

In [ ]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
from evaluate import load as load_metric

import warnings
warnings.filterwarnings("ignore")

2025-11-13 15:46:16.297562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763048776.489132      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763048776.545913      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# **THE DAILY MAIL DATASET**

In [ ]:

from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

train_data = dataset["train"]
validation_data = dataset["validation"]

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

## LOOKING OUT FOR A SAMPLE

## PRE PROCESSING FUNCTIONS

In [ ]:
SAMPLE_SIZE_TRAIN = 1000 
SAMPLE_SIZE_EVAL = 100 

train_data = train_data.select(range(SAMPLE_SIZE_TRAIN))
validation_data = validation_data.select(range(SAMPLE_SIZE_EVAL))

print(f"new data{len(train_data)}")
print(f"new data{len(validation_data)}")



In [ ]:
def format_simple_summarization(sample):
    return {
        "text": f"{sample['article']}\n\nSummary:\n{sample['highlights']}",
    }

def generate_instruction_dataset(data_point):
    
    formatted_data = format_simple_summarization(data_point)
    
    return {
        "article": data_point["article"],
        "highlights": data_point["highlights"],
        "text": formatted_data["text"] 
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(['id'])
    )

- **Converting the `article text` and `summary` to a `prompt`:**



```
    Summarize the following conversation.
    
    ### Input:
    (CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third 
    gold in Moscow as he anchored Jamaica to
    victory in the men's 4x100m relay. The fastest man in the world charged clear of United 
    States rival Justin Gatlin as the Jamaican
    quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 
    seconds. The U.S finished second in 37.56 seconds 
    with Canada taking the bronze after Britain were disqualified for a faulty handover. 
    The 26-year-old Bolt has n......
    

    ### Summary:

    Usain Bolt wins third gold of world championship .
    Anchors Jamaica to 4x100m relay victory .
    Eighth gold at the championships for Bolt .
    Jamaica double up in women's 4x100m relay .
```

In [ ]:

columns_to_remove = dataset['train'].column_names 


train_data = dataset['train'].shuffle(seed=42).select([i for i in range(1000)]).map(
    format_simple_summarization,
    remove_columns=columns_to_remove
)


validation_data = dataset['validation'].shuffle(seed=42).select([i for i in range(100)]).map(
    format_simple_summarization,
    remove_columns=columns_to_remove
)

test_data = dataset['test'].shuffle(seed=42).select([i for i in range(100)]) 


print(f"size of new data{len(train_data)}")
print(f"size of new data{len(validation_data)}")
print(f"size of new data {len(test_data)}")

print( train_data.column_names)
print( validation_data.column_names)
print(test_data.column_names)



# **THE LLAMA-2 7B MODEL**

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## ZERO-SHOT INFERENCE WITH LLAMA-2 7B

In [ ]:
index = 2


dialogue = test_data['article'][index] 
summary = test_data['highlights'][index] 


prompt = f"{dialogue}\n\nSummary:\n" 

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
New Jersey Governor Chris Christie wasn't looking too presidential Tuesday night when he got into a heated debate with a veteran teacher at a town hall meeting. And now the state's largest teacher's union is calling him out for his 'bullying' behavior. 'He's always taken a very nasty and disrespectful tone with teachers and other individuals who dare to question him at these events,' Steve Wollmer of the NJ Education Association told NJ.com. 'It's the one thing that never seems to change.' Scroll Down for Video . Not being nice: New Jersey Gov Chris Christie (left) is being called a bully for the way he interacted with a teacher (Kathy Mooney, right) at a Tuesday night town hall meeting . That sentiment doesn't ring well with Christie's ambitions to run in the Republican presidential primaries next year. Tuesday night, Christie appeared at Kenilworth Town Hall to take quest

# **TRAINING STEP (FINE TUNING)**

In [48]:
from peft import prepare_model_for_kbit_training

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [49]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [ ]:
from transformers import TrainingArguments

OUTPUT_DIR = "llama2-docsum-adapter_quick_test" 

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=1,  #
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  

In [ ]:


from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
13,1.808200,1.778207
26,1.769900,1.748653
39,1.707400,1.741484
52,1.746500,1.739321


TrainOutput(global_step=62, training_loss=1.7515673983481623, metrics={'train_runtime': 3923.3, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.016, 'total_flos': 1.774786419523584e+16, 'train_loss': 1.7515673983481623, 'epoch': 0.99})

In [ ]:


print("Train Data Columns:", train_data.column_names)
print("Validation Data Columns:", validation_data.column_names)

Train Data Columns: ['text']
Validation Data Columns: ['text']


In [54]:
peft_model_path="./peft-dialogue-summary"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary/tokenizer_config.json',
 './peft-dialogue-summary/special_tokens_map.json',
 './peft-dialogue-summary/tokenizer.model',
 './peft-dialogue-summary/added_tokens.json',
 './peft-dialogue-summary/tokenizer.json')

In [55]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False

In [ ]:
import os

peft_model_dir = "llama2-docsum-adapter_quick_test"

print(f"{peft_model_dir}")
try:
    
    print(os.listdir(peft_model_dir))
    
    if 'checkpoint-63' in os.listdir(peft_model_dir):
        print("Checkpoint-63:")
        print(os.listdir(os.path.join(peft_model_dir, 'checkpoint-63')))
except FileNotFoundError:
    print(f"{peft_model_dir} there iserror ")

In [ ]:


from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch


peft_model_dir = "llama2-docsum-adapter_quick_test/checkpoint-62" 


trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto",          
    local_files_only=True,      
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir, local_files_only=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
index = 2 


dialogue = dataset['test'][index]['article'] 
summary = dataset['test'][index]['highlights']


prompt = f"{dialogue}\n\nSummary:\n" 


input_ids = tokenizer(prompt, return_tensors='pt', truncation=True).input_ids.cuda()
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=500)


output = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]


output_summary = output[len(prompt):].strip()


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'TRAINED MODEL GENERATED TEXT :\n{output_summary}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
(CNN)If you've been following the news lately, there are certain things you doubtless know about Mohammad Javad Zarif. He is, of course, the Iranian foreign minister. He has been U.S. Secretary of State John Kerry's opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. And he received a hero's welcome as he arrived in Iran on a sunny Friday morning. "Long live Zarif," crowds chanted as his car rolled slowly down the packed street. You may well have read that he is "polished" and, unusually for one burdened with such weighty issues, "jovial." An Internet search for "Mohammad Javad Zarif" and "jovial" yields thousands of results. He certainly has gone a long way to bring Iran in from the cold and allow it to rejoin the international community. But there are

In [ ]:
index = 2


dialogue = dataset['test'][index]['article'] 
summary = dataset['test'][index]['highlights']


prompt = f"{dialogue}\n" 


input_ids = tokenizer(prompt, return_tensors='pt', truncation=True).input_ids.cuda()
outputs = trained_model.generate(
    input_ids=input_ids, 
    max_new_tokens=150,
    repetition_penalty=1.2 
)

output = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]


output_summary = output[len(prompt):].strip()

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'TRAINED MODEL GENERATED SUMMARY:\n{output_summary}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
(CNN)If you've been following the news lately, there are certain things you doubtless know about Mohammad Javad Zarif. He is, of course, the Iranian foreign minister. He has been U.S. Secretary of State John Kerry's opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. And he received a hero's welcome as he arrived in Iran on a sunny Friday morning. "Long live Zarif," crowds chanted as his car rolled slowly down the packed street. You may well have read that he is "polished" and, unusually for one burdened with such weighty issues, "jovial." An Internet search for "Mohammad Javad Zarif" and "jovial" yields thousands of results. He certainly has gone a long way to bring Iran in from the cold and allow it to rejoin the international community. But there are

In [74]:
print(output_summary )

Summary:
Zarif was educated abroad but returned home after the revolution;
He became involved in student protests while studying in California;
In 2007, he was named secretary of Iran's Supreme National Security Council;
He negotiates with Western nations on their nuclear program;
He is known for being polite and jocular despite difficult circumstances.

Source: CNN . Read entire article here . Summary: Zarif was educated abroad but returned home after the revolution; He became involved in student protests while studying in California; In 2007, he was named secretary of Iran's Supreme National Security Council; He negotiates with Western nations on their nuclear program; He is


In [ ]:

custom_article = """
Mohamed Salah was born in a small Egyptian village called Nagrig, where football was more than a game—it was a way of dreaming beyond the dusty streets and quiet fields. From a young age, he would run barefoot after a worn-out ball, imagining himself playing under bright stadium lights one day. His parents worried about his future, but Salah’s determination was stronger than anyone’s doubts. Every morning before school, he practiced his dribbling and speed, and after sunset, he stayed on the field until it was too dark to see. When he joined El Mokawloon, one of Egypt’s local clubs, his talent began to shine, but his journey was far from easy. He often took long bus rides—sometimes up to nine hours a day—to attend training, eating his meals on the road and sleeping on the way back. His coaches saw in him something rare: a hunger to prove himself. When Basel, a Swiss club, discovered him, it was the first time Salah left Egypt. He didn’t speak the language, he missed home terribly, but he kept reminding himself that his success could inspire millions back home. In Switzerland, his explosive pace and calm finishing quickly caught attention, and soon bigger clubs began to watch. Chelsea signed him, but his time there was frustrating—he barely played, struggled for confidence, and was sent out on loan. Yet Salah never lost his smile or his discipline. He used those difficult seasons to grow stronger, both mentally and physically. 
"""


prompt = f"{custom_article}\n" 


input_ids = tokenizer(prompt, return_tensors='pt', truncation=True).input_ids.cuda()
outputs = trained_model.generate(
    input_ids=input_ids, 
    max_new_tokens=150, 
    repetition_penalty=1.2 
)


output = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
output_summary = output[len(prompt):].strip()


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT ARTICLE (First 200 chars):\n{custom_article[:200]}...\n')
print(dash_line)
print(f'TRAINED MODEL GENERATED SUMMARY:\n{output_summary}')

---------------------------------------------------------------------------------------------------
INPUT ARTICLE (First 200 chars):

Mohamed Salah was born in a small Egyptian village called Nagrig, where football was more than a game—it was a way of dreaming beyond the dusty streets and quiet fields. From a young age, he would ru...

---------------------------------------------------------------------------------------------------
TRAINED MODEL GENERATED SUMMARY:
When Liverpool came calling with an offer worth £34 million ($50m), Salah knew this was his chance to make history. The Reds had won five European Cups since their formation in 1892; no player has ever scored as many goals in a single season (forty) as Salah did during the 2017–18 campaign. With every goal, he became part of the club's folklore. Now, at thirty years old, he is still hungry for more glory. This book tells the story behind these achievements, revealing how Salah overcame poverty, racism, injury, and self-doub

In [77]:
print(output_summary )

When Liverpool came calling with an offer worth £34 million ($50m), Salah knew this was his chance to make history. The Reds had won five European Cups since their formation in 1892; no player has ever scored as many goals in a single season (forty) as Salah did during the 2017–18 campaign. With every goal, he became part of the club's folklore. Now, at thirty years old, he is still hungry for more glory. This book tells the story behind these achievements, revealing how Salah overcame poverty, racism, injury, and self-doubt to become one of the world'


In [44]:
!zip -r model_weights.zip llama2-docsum-adapter_quick_test/checkpoint-62

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	zip warning: name not matched: llama2-docsum-adapter_quick_test/checkpoint-62

zip error: Nothing to do! (try: zip -r model_weights.zip . -i llama2-docsum-adapter_quick_test/checkpoint-62)
